In [4]:
import sys
sys.path.insert(1, '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/Deep-Opt')

In [5]:
import torch
import numpy as np

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

In [6]:
# Highly recommended to keep as cpu for problems of size <= 100
device="cpu"
print(device)
device = torch.device(device)

cpu


In [7]:
durations = [13, 23, 13, 13, 13, 13, 13, 13, 23, 13, 10, 10, 10, 20, 25, 20, 10, 15, 10, 20]
# durations = [13, 23,16, 20]

## problem converter

### 1

In [169]:
def generate_qubo_matrix(durations):
    """
    Generate a QUBO matrix for the given job durations.

    Parameters:
    durations (list): List of job durations.

    Returns:
    torch.Tensor: The QUBO matrix as a PyTorch tensor.
    """
    # Number of jobs
    n = len(durations)

    # Initialize QUBO matrix
    Q = np.zeros((n, n))

    # Fill the QUBO matrix
    for i in range(n):
        for j in range(i, n):
            if i == j:
                Q[i, j] = durations[i] ** 2
            else:
                Q[i, j] = 2 * durations[i] * durations[j]

    # Convert Q matrix to PyTorch tensor and negate for minimization
    Q = -Q
    Q_tensor = torch.tensor(Q)

    return Q_tensor

In [170]:
Q = generate_qubo_matrix(durations)

In [171]:
Q

tensor([[ -169.,  -598.,  -338.,  -338.,  -338.,  -338.,  -338.,  -338.,  -598.,
          -338.,  -260.,  -260.,  -260.,  -520.,  -650.,  -520.,  -260.,  -390.,
          -260.,  -520.],
        [   -0.,  -529.,  -598.,  -598.,  -598.,  -598.,  -598.,  -598., -1058.,
          -598.,  -460.,  -460.,  -460.,  -920., -1150.,  -920.,  -460.,  -690.,
          -460.,  -920.],
        [   -0.,    -0.,  -169.,  -338.,  -338.,  -338.,  -338.,  -338.,  -598.,
          -338.,  -260.,  -260.,  -260.,  -520.,  -650.,  -520.,  -260.,  -390.,
          -260.,  -520.],
        [   -0.,    -0.,    -0.,  -169.,  -338.,  -338.,  -338.,  -338.,  -598.,
          -338.,  -260.,  -260.,  -260.,  -520.,  -650.,  -520.,  -260.,  -390.,
          -260.,  -520.],
        [   -0.,    -0.,    -0.,    -0.,  -169.,  -338.,  -338.,  -338.,  -598.,
          -338.,  -260.,  -260.,  -260.,  -520.,  -650.,  -520.,  -260.,  -390.,
          -260.,  -520.],
        [   -0.,    -0.,    -0.,    -0.,    -0.,  -169.,  -3

### 2

In [172]:
def generate_qubo_matrix(arr):
    n = len(arr)
    A = sum(arr)
    Q = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            if i == j:
                Q[i, j] = 4 * arr[i] ** 2 - 4 * A * arr[i]
            else:
                Q[i, j] = Q[j, i] = 8 * arr[i] * arr[j]
    
    return Q

In [173]:
Q = generate_qubo_matrix(durations)

In [174]:
Q

array([[-14924.,   2392.,   1352.,   1352.,   1352.,   1352.,   1352.,
          1352.,   2392.,   1352.,   1040.,   1040.,   1040.,   2080.,
          2600.,   2080.,   1040.,   1560.,   1040.,   2080.],
       [  2392., -25484.,   2392.,   2392.,   2392.,   2392.,   2392.,
          2392.,   4232.,   2392.,   1840.,   1840.,   1840.,   3680.,
          4600.,   3680.,   1840.,   2760.,   1840.,   3680.],
       [  1352.,   2392., -14924.,   1352.,   1352.,   1352.,   1352.,
          1352.,   2392.,   1352.,   1040.,   1040.,   1040.,   2080.,
          2600.,   2080.,   1040.,   1560.,   1040.,   2080.],
       [  1352.,   2392.,   1352., -14924.,   1352.,   1352.,   1352.,
          1352.,   2392.,   1352.,   1040.,   1040.,   1040.,   2080.,
          2600.,   2080.,   1040.,   1560.,   1040.,   2080.],
       [  1352.,   2392.,   1352.,   1352., -14924.,   1352.,   1352.,
          1352.,   2392.,   1352.,   1040.,   1040.,   1040.,   2080.,
          2600.,   2080.,   1040.,   1

### 3

In [8]:
def generate_qubo_matrix(S):
    n = len(S)
    c = sum(S)
    Q = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i == j:
                Q[i, j] = S[i] * (S[i] - c)
            else:
                Q[i, j] = S[i] * S[j]
    
    return Q

In [9]:
Q = generate_qubo_matrix(durations)

In [10]:
Q

array([[-3731.,   299.,   169.,   169.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  299., -6371.,   299.,   299.,   299.,   299.,   299.,   299.,
          529.,   299.,   230.,   230.,   230.,   460.,   575.,   460.,
          230.,   345.,   230.,   460.],
       [  169.,   299., -3731.,   169.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169., -3731.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169.,   169., -3731.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169.,   169.,   169., -3731.,   169.,   169.,
   

## save problem

In [47]:
# Generate the text file content
lines = []
lines.append("1\n")  # Number of problem instances

# Collect non-zero entries
n = Q.shape[0]
non_zero_entries = []
for i in range(n):
    for j in range(i, n):
        if Q[i][j] != 0:
            non_zero_entries.append((i + 1, j + 1, int(Q[i][j])))

# Add the number of variables and number of non-zero entries
lines.append(f"{n} {len(non_zero_entries)}\n")

# Add the non-zero entries to lines
for entry in non_zero_entries:
    i, j, value = entry
    lines.append(f"{i} {j} {value}\n")

# Write to file
with open("../data/qubo/tpp_qubo20.txt", "w") as f:
    f.writelines(lines)

## load problem

In [126]:
problem = QUBO("../data/qubo/qubo_problem_small.txt", 0, device)

Instance has been loaded


In [127]:
problem

In [128]:
problem.Q

tensor([[-3731.,   299.,   169.,   169.,   169.,   169.,   169.,   169.,   299.,
           169.,   130.,   130.,   130.,   260.,   325.,   260.,   130.,   195.,
           130.,   260.],
        [  299., -6371.,   299.,   299.,   299.,   299.,   299.,   299.,   529.,
           299.,   230.,   230.,   230.,   460.,   575.,   460.,   230.,   345.,
           230.,   460.],
        [  169.,   299., -3731.,   169.,   169.,   169.,   169.,   169.,   299.,
           169.,   130.,   130.,   130.,   260.,   325.,   260.,   130.,   195.,
           130.,   260.],
        [  169.,   299.,   169., -3731.,   169.,   169.,   169.,   169.,   299.,
           169.,   130.,   130.,   130.,   260.,   325.,   260.,   130.,   195.,
           130.,   260.],
        [  169.,   299.,   169.,   169., -3731.,   169.,   169.,   169.,   299.,
           169.,   130.,   130.,   130.,   260.,   325.,   260.,   130.,   195.,
           130.,   260.],
        [  169.,   299.,   169.,   169.,   169., -3731.,   1

In [130]:
sum([25, 7,13, 31, 42,17, 21,10])

166

In [132]:
166**2

27556

In [129]:
Q

array([[-3731.,   299.,   169.,   169.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  299., -6371.,   299.,   299.,   299.,   299.,   299.,   299.,
          529.,   299.,   230.,   230.,   230.,   460.,   575.,   460.,
          230.,   345.,   230.,   460.],
       [  169.,   299., -3731.,   169.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169., -3731.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169.,   169., -3731.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169.,   169.,   169., -3731.,   169.,   169.,
   

In [114]:
np.triu(Q)

array([[-3525.,   175.,   325.,   775.,  1050.,   425.,   525.,   250.],
       [    0., -1113.,    91.,   217.,   294.,   119.,   147.,    70.],
       [    0.,     0., -1989.,   403.,   546.,   221.,   273.,   130.],
       [    0.,     0.,     0., -4185.,  1302.,   527.,   651.,   310.],
       [    0.,     0.,     0.,     0., -5208.,   714.,   882.,   420.],
       [    0.,     0.,     0.,     0.,     0., -2533.,   357.,   170.],
       [    0.,     0.,     0.,     0.,     0.,     0., -3045.,   210.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0., -1560.]])

In [118]:
change_tolerance = Q.shape[0]
problem_size = Q.shape[0]
pop_size = 100

dropout_prob = 0.2
# 1000 bit QUBO l1 and l2:
# l1_coef = 0.0000025
# l2_coef = 0.0000025
# 500 bit QUBO l1 and l2:
# l1_coef = 0.000005
# l2_coef = 0.000005
# 100 bit l1 and l2
l1_coef = 0.0001
l2_coef = 0.0001
lr = 0.002
compression_ratio = 0.8
model = DOAE(problem_size, dropout_prob, device)
hidden_size = problem_size
handler = OptimAEHandler(model, problem, device)

population, fitnesses = handler.generate_population(pop_size)
population, fitnesses, _, _ = handler.hillclimb(population, fitnesses, change_tolerance)
handler.print_statistics(fitnesses)

total_eval = 0
max_depth = 6
depth = 0

while True:
    if depth < max_depth:
        hidden_size = round(hidden_size * compression_ratio)
        model.transition(hidden_size)
        depth += 1
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_coef)
    print("Learning from population")
    # Learing with the entire population as a batch is technically not the best from a machine learning perspective,
    # but does not seem to have a massive impact on solution quality whilst also increasing learning speed significantly.
    handler.learn_from_population(population, optimizer, l1_coef=l1_coef, batch_size=pop_size)
    print("Optimising population")
    population, fitnesses, evaluations, done = handler.optimise_solutions(
        population, fitnesses, change_tolerance, encode=True, repair_solutions=True, deepest_only=False
    )
    handler.print_statistics(fitnesses)
    total_eval += evaluations
    print("Evaluations: {}".format(total_eval))
    
    # Uncomment lines below to print out best solution at every transition
    # best_i = torch.argmax(fitnesses)
    # print("Best solution - fitness = {}".format(fitnesses[best_i].item()))
    # print(population[best_i].tolist())
    if done:
        break

Max pop fitness: 0.0, Mean pop fitness : -694.02001953125
Learning from population
Optimising population
Max pop fitness: 0.0, Mean pop fitness : -588.2000122070312
Evaluations: 920
Learning from population
Optimising population
Max pop fitness: 0.0, Mean pop fitness : -186.89999389648438
Evaluations: 2772
Learning from population
Optimising population
Max pop fitness: 0.0, Mean pop fitness : -104.0199966430664
Evaluations: 5481
Learning from population
Optimising population
Max pop fitness: 0.0, Mean pop fitness : -55.380001068115234
Evaluations: 9086
Learning from population
Optimising population
Max pop fitness: 0.0, Mean pop fitness : 0.0
Evaluations: 13589
Learning from population
Optimising population
Max pop fitness: 0.0, Mean pop fitness : 0.0
Evaluations: 18989
Learning from population
Optimising population
Max pop fitness: 0.0, Mean pop fitness : 0.0
Evaluations: 24389
Learning from population
Optimising population
Max pop fitness: 0.0, Mean pop fitness : 0.0
Evaluations: 297

KeyboardInterrupt: 

In [119]:
population[0]

tensor([-1., -1., -1., -1., -1., -1., -1., -1.])

In [77]:
fitnesses

tensor([4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508.,
        4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508.,
        4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508.,
        4508., 4508., 4508., 4508., 4508., 4508., 4508., 3324., 4508., 4508.,
        4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508.,
        4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508.,
        4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508.,
        4508., 4508., 4508., 3068., 4508., 4508., 4508., 4508., 4508., 4508.,
        4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508.,
        4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508., 4508.])

In [78]:
population[1]

tensor([-1.,  1., -1., -1.])

In [97]:
torch.count_nonzero(problem.Q, dim=0), torch.count_nonzero(problem.Q, dim=1)

(tensor([4, 4, 4, 4]), tensor([4, 4, 4, 4]))

In [98]:
torch.count_nonzero(Q_tensor, dim=0), torch.count_nonzero(Q_tensor, dim=1)

(tensor([1, 2, 3, 4]), tensor([4, 3, 2, 1]))

In [99]:
Q_tensor[1]

tensor([  -0., -529., -736., -920.], dtype=torch.float64)

In [100]:
problem.Q[1]

tensor([-598., -529., -736., -920.])